# install extra library

In [1]:
!pip install transformers torch
!pip install sentencepiece

  Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 976.6 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.4 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 4.8 MB/s eta 0:00:0000:0100:01
  Using cached sentencepiece-0.1.97-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)


# read stored data from DataPlatform

In [3]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [4]:
dir_raw_id = 64
dir_process_id = 65

In [5]:
datanode = DataNode()

Init DataNode sucessful!


In [6]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='kaoded.json'))
meta

{'owner': 'suphawadi',
 'human_size': '67.03 KB',
 'name': 'kaoded.json',
 'description': 'file kaoded.json',
 'directory': 64,
 'path': 'suphawadi/DSI310-12/rawData/kaoded.json'}

In [7]:
data = json.load(fp)
data

[{'topic_acc': 'สงขลา ปาดังเบซาร์-ท่าข่อย ถนนพังเป็นหลุมใหญ่ เกิดอุบัติเหตุหลายครั้ง หน่วยงานฯเมิน',
  'date_published': '07 พ.ย. 2022',
  'province_acc': 'สงขลา',
  'link_url': 'https://www.77kaoded.com/news/phumarin/2365601'},
 {'topic_acc': 'เพชรบูรณ์-ผญบ.ทับเบิกเบื่อทนฟัง 9 ปีอยู่ในเขตป่า นำชาวบ้านเดินหน้าลุยซ่อมถนน โดยไม่ใช้งบฯราชการ หลังนทท.เกิดอุบัติเหตุบ่อย',
  'date_published': '02 พ.ย. 2022',
  'province_acc': 'เพชรบูรณ์',
  'link_url': 'https://www.77kaoded.com/news/soontorn/2362961'},
 {'topic_acc': 'อุบัติเหตุสยอง!! ปิคอัพชน จยย.พ่วงข้างอย่างจัง ร่างปลิวตกสะพานข้ามคลองพระสทึงจมดับ',
  'date_published': '29 ต.ค. 2022',
  'province_acc': 'สระแก้ว',
  'link_url': 'https://www.77kaoded.com/news/thanapat/2361453'},
 {'topic_acc': 'ลำปางรณรงค์ลดอุบัติเหตุทางข้ามเคร่งครัดบังคับใช้กฎหมาย พ.ร.บ.จราจรทางบก',
  'date_published': '26 ต.ค. 2022',
  'province_acc': 'ลำปาง',
  'link_url': 'https://www.77kaoded.com/news/prateep/2360276'},
 {'topic_acc': 'นครนายก- (มีคลิป) อุบัติเหตุรถยนต์

# NER (NameEntitiesRegcognition)

In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/211 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/980 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [10]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [15]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [19]:
text = data[10]['topic_acc']
text

'นครนายก-อุบัติเหตุรถเก๋งพุ่งชน รถจยย. คนขี่รถจยย. ลอยไปกระแทกเสาฟ้า ดับคาที่'

In [20]:
import re

In [21]:
output = nlp(text)
output

[{'entity': 'B-LOC',
  'score': 0.9997987,
  'index': 1,
  'word': '▁',
  'start': 0,
  'end': 1},
 {'entity': 'B-LOC',
  'score': 0.9985915,
  'index': 2,
  'word': 'นคร',
  'start': 0,
  'end': 3},
 {'entity': 'I-LOC',
  'score': 0.99967444,
  'index': 3,
  'word': 'นาย',
  'start': 3,
  'end': 6},
 {'entity': 'I-LOC',
  'score': 0.99967825,
  'index': 4,
  'word': 'ก',
  'start': 6,
  'end': 7}]

In [22]:
post_process(output)

[{'type': 'B-LOC', 'text': ' '},
 {'type': 'I-LOC', 'text': 'นครนายก'},
 {'type': 'I-LOC', 'text': 'นายก'},
 {'type': 'I-LOC', 'text': 'ก'}]

# Process news to spo

In [23]:
spo = []
for kaoded in tqdm(data):
    topic_acc = kaoded.get('topic_acc') 
    if topic_acc == None:
        continue
    spo.append({
        'topic_acc': kaoded.get('topic_acc', ""),
        'date_published': kaoded.get('date_published', ""),
        'province_acc': kaoded.get('province_acc', ""),
        'link_url': kaoded.get('link_url', ""),
        })

  0%|          | 0/110 [00:00<?, ?it/s]

In [24]:
df = pd.DataFrame(spo)
df

,topic_acc,date_published,province_acc,link_url
0,สงขลา ปาดังเบซาร์-ท่าข่อย ถนนพังเป็นหลุมใหญ่ เ...,07 พ.ย. 2022,สงขลา,https://www.77kaoded.com/news/phumarin/2365601
1,เพชรบูรณ์-ผญบ.ทับเบิกเบื่อทนฟัง 9 ปีอยู่ในเขตป...,02 พ.ย. 2022,เพชรบูรณ์,https://www.77kaoded.com/news/soontorn/2362961
2,อุบัติเหตุสยอง!! ปิคอัพชน จยย.พ่วงข้างอย่างจัง...,29 ต.ค. 2022,สระแก้ว,https://www.77kaoded.com/news/thanapat/2361453
3,ลำปางรณรงค์ลดอุบัติเหตุทางข้ามเคร่งครัดบังคับใ...,26 ต.ค. 2022,ลำปาง,https://www.77kaoded.com/news/prateep/2360276
4,นครนายก- (มีคลิป) อุบัติเหตุรถยนต์เสียหลักพุ่ง...,09 ต.ค. 2022,นครนายก,https://www.77kaoded.com/news/ibaspea/2352400
...,...,...,...,...
105,เพชรบูรณ์-เกิดอุบัติเหตุซ้อน รถพยาบาลฉุกเฉิน ช...,14 ม.ค. 2022,เพชรบูรณ์,https://www.77kaoded.com/news/soontorn/2226908
106,ชายอายุราว 50 ปี เกิดอุบัติเหตุรถกระบะตกในร่อง...,14 ม.ค. 2022,เชียงราย,https://www.77kaoded.com/news/big/2226700
107,(ชมคลิป) ยูเทิร์นวัดใจ เจ็บตายบ่อย เกิดอุบัติเ...,10 ม.ค. 2022,เชียงราย,https://www.77kaoded.com/news/big/2224781
108,พลิกตะแคงหวิดดับ อุบัติเหตุรถยนต์ชนสนั่น 3 คัน...,09 ม.ค. 2022,ปราจีนบุรี,https://www.77kaoded.com/news/thanapat/2224141


In [25]:
datanode.write(df=df, directory=dir_process_id, name="week03", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='kaoded.json')])

{'sucess': True,
 'file_id': 513,
 'path': 'suphawadi/DSI310-12/processData/week03.parquet',
 'lineage': True}